<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized%5D_%5BLocal_Runtime%5D22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [ ]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
# !pip install swmm5
!pip install openpyxl
import math
import itertools
import os, sys
# import swmm5
from pyswmm import Simulation, Links
# from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
# from google.colab import files
import pandas as pd
import re
import copy
import glob
import bisect
import multiprocessing

# File Upload

In [ ]:
# uploaded = files.upload()

inp = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp")
# inp = SwmmInput(r"C:\Users\Ferdz\Desktop\Harvey_Files\Thesis\Study_Model.inp")
# inp = SWMMInput("Study_Model.inp")
# available_sizes = uploaded["Available Sizes.txt"]
available_sizes = r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Available_Sizes.txt"
# available_sizes = r"C:\Users\Ferdz\Desktop\Harvey_Files\Thesis\Available_Sizes.txt"
# pipe_prices = uploaded["Pipe_Prices.xlsx"]
pipe_prices = r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Pipe_Prices.txt"
# print("Uploaded files:", os.listdir())
# conduits_df = inp["CONDUITS"]
# print(conduits_df)

# Boundary Parameters

In [ ]:
vel_min = 0.8
vel_max = 5
slope_min = 0.01
mannings_n = 0.013
peak_flow_dict = {}
peak_flow_results =  {}
peak_vel = {}
adjust_pipe_size = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}
trials = {}
velocity = {}
largest_size = {}
smallest_size = {}

# Data Frame

In [ ]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']
junctions_data = inp['JUNCTIONS']
outfalls_data = inp['OUTFALLS']

# Store results
conduit_info = []

In [ ]:
for junction_name, junction_data in inp["JUNCTIONS"].items():

  junction_depth = junction_data.depth_max
  print(f"Junction Name: {junction_name}")
  print(f"Junction Depth: {junction_depth}")
  print(f"")

# for conduit_name, conduit_data in inp["CONDUITS"].items():
#   conduit_dict = vars(conduit_data) # Access all data in Conduit
#   # print(conduit_dict)

# print(xsections)  # Print the content of the xsections InpSection object
# for xsection_name, xsection_data in xsections.items():
# # print(xsection_data)  # Print the name of each parameter in the xsections section

Test 1

In [ ]:
with Simulation(r"Optimized_Model2.inp") as sim:
    links = Links(sim)  # Create a Links object
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid  # Get conduit name (link ID)
      conduit_data = link.conduit_statistics  # The 'link' object itself holds the conduit data

      peak_flow = conduit_data["peak_flow"]
      peak_depth = conduit_data.get("peak_depth")

      # Check if peak_depth is not None and is a sequence (list, tuple, etc.)
      if peak_depth is not None and isinstance(peak_depth, (list, tuple)):
      # Use peak_depth[0] if it's a sequence
          actual_peak_depth = peak_depth[0]
      else:
          # Use peak_depth directly if it's a single value
          actual_peak_depth = peak_depth

      theta = 2 * math.acos(1 - (2 * actual_peak_depth / 0.3))
      hydraulic_radius = (0.3/4) * (1 - math.sin(theta)/theta)

      # # If you want to store it in a dictionary:
      # peak_flow_dict = {}  # Initialize an empty dictionary if needed
      # peak_flow_dict[conduit_name] = peak_flow
      print(f"Conduit Name: {conduit_name}")
      print(f"Peak Flow: {peak_flow}")
      print(f"Peak Depth: {peak_depth}")
      print(f"Actual Depth: {actual_peak_depth}")
      print(f"R: {hydraulic_radius}")  # Accessing a property of the conduit
      # Access other properties like link.roughness, link.inlet_offset, etc.

# Data Gathering

In [ ]:
with Simulation(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp") as sim:
    links = Links(sim)
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid
      conduit_data = link.conduit_statistics
      peak_flow = conduit_data["peak_flow"]

      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {peak_flow}")

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    name = xsection_name
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

    print (f"Name: {name}")
    print (f"Length: {length} m")
    print (f"Shape: {shape}")
    print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
    print (f"Inlet Offset: {in_offset} m")
    print (f"Outlet Offset: {out_offset} m")
    print (f"")

# Arrange Conduits based on Pipe Flow


In [ ]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths


flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths

# Show results
for outfall_node, flow_path in flow_paths.items():
    print(f"All Flow Paths for {outfall_node}: {flow_path}")  # Print all paths for the outfall
    for path in flow_path:  # Iterate through each path in flow_path
        path.reverse()  # Reverse each individual path
        print(f"Flow Path: {path}")

        # Modified logic to identify conduits using node names
        conduits = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]

            # Find the conduit connecting these nodes
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits.append(conduit_name)
                    break  # Stop searching once the conduit is found

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits}")
        print(f"Junctions in Path: {junctions}")

# Identify the Invert Elevations of the Conduits

In [ ]:
conduit_info = []
for conduit_name, conduit_data in inp['CONDUITS'].items():
    upstream_junction = conduit_data.from_node
    downstream_junction = conduit_data.to_node

    upstream_invert = junctions_data[upstream_junction].elevation

    # Check if downstream node is an outfall
    if downstream_junction in outfalls_data:
        downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
    elif downstream_junction in junctions_data:
        downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
    else:
        print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
        downstream_invert = None  # or some default value

    length = conduit_data.length

    if downstream_invert is not None:
        slope = (upstream_invert - downstream_invert) / length
    else:
        slope = None
    # Input all data gathered and solve in a DataFrame
    conduit_info.append([conduit_name, upstream_junction, upstream_invert, downstream_junction, downstream_invert, length, slope])

    print(f"Conduit: {conduit_name}; From: {upstream_junction} To: {downstream_junction}")
    print(f"Upstream Invert Elevation: {upstream_invert} m")
    print(f"Downstream Invert Elevation: {downstream_invert} m")
    print(f"Slope: {slope}")

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Upstream Invert', 'To Node', 'Downstream Invert', 'Length', 'Slope'])

conduit_df.to_excel("conduit_analysis_data.xlsx", index=False)



# Function: Identifying Peak Flow Rates

In [ ]:
def peak_flow_solve(inp, conduit_name):
  with Simulation(inp) as sim:
      links = Links(sim)
      for step in sim:
          pass

      # Iterate through links to find the specific conduit
      for link in links:
          if link.linkid == conduit_name:
              conduit_data = link.conduit_statistics
              peak_flow = conduit_data.get("peak_flow")  # Use get() to handle missing keys
              if peak_flow is not None and peak_flow != 0.0:
                  return float(peak_flow)  # Convert to float if found
              else:
                  return 0.0  # Or handle the missing peak_flow case differently
      # If the conduit is not found
      return 0.0  # Or handle the missing conduit case differently

# Function: Identifying the Largest Pipe Size

In [ ]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size_available(available_sizes):
  largest_size = 0.0
  with open(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Available_Sizes.txt") as file:
  # with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

largest_size = largest_size_available(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

# Function: Identifying the Smallest Pipe Size

In [ ]:
# Identify the smallest pipe size available
def smallest_size_available(available_sizes):
  largest_size = largest_size_available(available_sizes)
  smallest_size = largest_size
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size < largest_size:
          smallest_size = size
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return smallest_size

# Call the function and assign the result to a variable
smallest_size = smallest_size_available(available_sizes)

print(f"Smallest Pipe Size: {smallest_size} m")

# Function: Find Velocities per Pipe Size

In [ ]:
def calculate_velocities(conduit_name, available_sizes, peak_flow):
    """
    Calculates and stores velocities for each conduit and available size combination.

    Args:
        conduit_name (str): The name of the conduit.
        available_sizes (str): Path to the file containing available pipe sizes.
        peak_flow (float): The peak flow rate for the conduit.

    Returns:
        dict: A dictionary where keys are conduit names and values are dictionaries
              containing velocities for each available size.
    """

    velocities = {}  # Initialize a dictionary to store velocities

    if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
        available_sizes_list = []
        with open(available_sizes) as file:
            for line in file:
                try:
                    size = float(line.strip())
                    available_sizes_list.append(size)
                except ValueError:
                    print(f"Skipping invalid line: {line.strip()}")

        # Store velocities for each available size
        velocities[conduit_name] = {}
        for size in available_sizes_list:
            velocity = peak_flow / (math.pi * (size ** 2) * 0.25)
            velocities[conduit_name][size] = velocity
            print(f"Conduit: {conduit_name}, Size: {size}, Velocity: {velocity} m/s")

    return velocities

# Function: Identifying Suitable Pipe Sizes

In [ ]:
def find_suitable_sizes_with_velocity(all_velocities, vel_min, vel_max):
    """
    Finds suitable pipe sizes based on calculated velocities and constraints.

    Args:
        all_velocities (dict): Dictionary containing velocities per conduit and size.
        vel_min (float): Minimum allowable velocity.
        vel_max (float): Maximum allowable velocity.

    Returns:
        dict: Dictionary containing suitable sizes for each conduit.
    """
    suitable_sizes_per_conduit = {}  # Store suitable sizes here

    for conduit_name, size_velocity_dict in all_velocities.items():
        suitable_sizes = []  # Sizes within allowable velocity range for this conduit

        for size, velocity in size_velocity_dict.items():
            if vel_min <= velocity <= vel_max:
                if not suitable_sizes or size < suitable_sizes[0]:  # Check if smaller or list is empty
                    suitable_sizes = [size]  # Replace with smaller size
                elif size == suitable_sizes[0]:  # Handle equal sizes (optional)
                    suitable_sizes.append(size)  # You might want to append if sizes are equal

        # Handle cases where no suitable sizes were found:
        if not suitable_sizes:
            if all(velocity < vel_min for velocity in size_velocity_dict.values()):
                # All velocities below minimum, use largest available size
                suitable_sizes.append(min(size_velocity_dict.keys()))
            elif all(velocity > vel_max for velocity in size_velocity_dict.values()):
                # All velocities above maximum, use smallest available size
                suitable_sizes.append(max(size_velocity_dict.keys()))

        suitable_sizes_per_conduit[conduit_name] = suitable_sizes
        print(f"Conduit Name: {conduit_name}, Suitable Sizes: {suitable_sizes}")
    return suitable_sizes_per_conduit

# Function: Solving for the Max Hydraulic Radius

In [ ]:
def get_max_hydraulic_radius(inp_file, conduit_name, diameter):
    """
    Gets the maximum hydraulic radius for a given conduit.

    Args:
        inp_file (str): Path to the SWMM input file.
        conduit_name (str): Name of the conduit to analyze.

    Returns:
        float: The maximum hydraulic radius for the conduit.
    """

    with Simulation(inp_file) as sim:
        links = Links(sim)
        for step in sim:
            pass  # Advance the simulation to the next time step

        # Iterate through links to find the specific conduit
        for link in links:
            if link.linkid == conduit_name:
                # Get the flow and depth for the current link and timestep
                conduit_data = link.conduit_statistics
                peak_depth = conduit_data.get("peak_depth")

                # Check if peak_depth is not None and is a sequence (list, tuple, etc.)
                if peak_depth is not None and isinstance(peak_depth, (list, tuple)):
                    # Use peak_depth[0] if it's a sequence
                    actual_peak_depth = peak_depth[0]
                else:
                    # Use peak_depth directly if it's a single value
                    actual_peak_depth = peak_depth
                if actual_peak_depth > diameter:
                    actual_peak_depth = diameter
                else:
                    actual_peak_depth = actual_peak_depth

                theta = 2 * math.acos(1 - (2 * actual_peak_depth / diameter))
                hydraulic_radius = (diameter / 4) * (1 - math.sin(theta)/theta)

    return hydraulic_radius

# Function: Manning's Equation

In [ ]:
def adjust_slope_with_mannings(new_diameter, peak_flow, mannings_n, max_hydraulic_radius):

    # # Load the input file using SwmmInput
    # modified_inp = SwmmInput(input_file)

    # Get peak flow rate (Q) for the conduit - you'll need to adapt this part
    # based on how you are getting peak flow rates in your code
    # peak_flow = peak_flow_solve(input_file, conduit_name) # or use your similar function
    # peak_flow = get_peak_flow(modified_inp, conduit_name)
    # peak_flow = current_peak_flows[conduit_name]

    # Calculate cross-sectional area and hydraulic radius for the new diameter
    area = math.pi * (new_diameter / 2)**2

    # Calculate the new slope (S) using Manning's equation
    new_slope = (peak_flow * mannings_n / (area * max_hydraulic_radius**(2/3)))**2

    # Return the modified SwmmInput object
    return new_slope

# Function: Identifying Slopes

In [ ]:
def adjusting_slopes(modified_inp, conduit_name, pipe_size, is_last_conduit):

    # Get upstream and downstream junction names for the conduit
    upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
    downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

    # Handle the last conduit differently
    if is_last_conduit:
        # Use original downstream elevation from OUTFALLS section
        new_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
    else:
        # Otherwise, get downstream elevation from JUNCTIONS section
        new_downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

    # Calculate the new upstream invert elevation using Manning's equation and the optimized size
    new_slope = adjust_slope_with_mannings(modified_inp, conduit_name, pipe_size)
    conduit_length = modified_inp["CONDUITS"][conduit_name].length
    new_upstream_elevation = new_downstream_elevation + (new_slope * conduit_length)

    return new_upstream_elevation

# Step 1: Initialize All Pipe Sizes and Slopes

In [ ]:
modified_inp = copy.deepcopy(inp)

# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths


# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits_in_path}")  # Print sorted conduits
        print(f"Junctions in Path: {junctions}")  # Print junctions in path

        # Reverse the conduits_in_path list before processing
        conduits_in_path.reverse()

        current_downstream_elevation = None  # Initialize for the first (last) conduit

        for conduit_name in conduits_in_path:  # Iterate in the correct order

            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

            # Always use slope_min, regardless of original slope
            slope = slope_min

            # Find the xsection associated with this conduit
            xsection_name = None
            for xs_name, xs_data in xsections.items():
                if xs_data.link == conduit_name:
                    xsection_name = xs_name
                    break

            # Assign the largest size if xsection is found
            if xsection_name is not None:
                xsection_data = modified_inp["XSECTIONS"][xsection_name]

                if callable(largest_size_available):
                    conduit_size = largest_size_available(available_sizes)
                else:
                    conduit_size = largest_size_available

                if xsection_data.shape == "CIRCULAR":
                    xsection_data.height = conduit_size

            # If it's the first conduit (last in the original path),
            # use original downstream elevation from OUTFALLS or JUNCTIONS
            if conduit_name == conduits_in_path[0]:
                if downstream_junction in modified_inp["OUTFALLS"]:
                    current_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
                else:
                    current_downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

            # Otherwise, use the calculated upstream elevation of the previous conduit
            # as the current downstream elevation
            else:
                current_downstream_elevation = new_upstream_elevation  # Use the previously calculated value

            # Calculate new upstream elevation based on slope and current downstream elevation
            new_upstream_elevation = current_downstream_elevation + (slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            modified_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            print(f"Name: {xsection_name}")
            print(f"Shape: {xsection_data.shape}")
            print(f"Conduit Height: {xsection_data.height}")
            print(f"Downstream Elevation: {current_downstream_elevation}")
            print(f"Upstream Elevation: {new_upstream_elevation}")
            print(f"")

# Create a unique filename to avoid overwriting
file_name = f"Modified_Model1.inp"
modified_inp.write_file(file_name)
modified_inp = SwmmInput("Modified_Model1.inp")
print(f"File saved as: {file_name}")

# Step 2: Optimize Pipe Sizes

Starting Pipe Size Optimization

In [ ]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits (modified to handle branches)
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit, considering branches
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Explore each branch

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths

# Initialize all_starting_pipes before using it in the loop
all_starting_pipes = []

# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits_in_path}")  # Print sorted conduits
        print(f"Junctions in Path: {junctions}")  # Print junctions in path

        # Check if any conduits were found in this path and it's not empty
        if conduits_in_path and conduits_in_path[0]:
            starting_pipe = conduits_in_path[0]  # Get the first conduit as the starting pipe
            if starting_pipe not in all_starting_pipes:  # Check for duplicates
                all_starting_pipes.append(starting_pipe)

print(f"All Starting Pipes: {all_starting_pipes}")  # Print the final list of starting pipes

# Get suitable sizes for starting pipes ONLY
suitable_sizes_per_pipe = {}
all_velocities = {}
all_suitable_sizes = {}
for conduit_name in all_starting_pipes:
    # Ensure conduit_name is a string
    conduit_name_str = str(conduit_name)

    # Get peak flow for the conduit
    peak_flow_value = peak_flow_solve("Modified_Model1.inp", conduit_name_str)
    print(peak_flow_value)

    # Get suitable sizes based on the peak flow value
    velocities_per_pipe = calculate_velocities(conduit_name_str, available_sizes, peak_flow_value)
    all_velocities.update(velocities_per_pipe)

    #Identify suitable sizes per pipe
    suitable_sizes_per_pipe = find_suitable_sizes_with_velocity(all_velocities, vel_min, vel_max)
    all_suitable_sizes.update(suitable_sizes_per_pipe)

Processing outfall node: OF-X1
Sorted Conduits: ['44', '45', '46', '91']
Junctions in Path: ['X1', 'X2', 'X3', 'X4', 'OF-X1']
Processing outfall node: OF-X2
Sorted Conduits: ['39', '386', '387', '388', '40', '41', '42', '43']
Junctions in Path: ['X21/1', 'X21/2', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'OF-X2']
Processing outfall node: OF-W1
Sorted Conduits: ['75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '93']
Junctions in Path: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'OF-W1']
Processing outfall node: OF-W2
Sorted Conduits: ['64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '92']
Junctions in Path: ['W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', 'W24', 'W25', 'W26', 'W27', 'OF-W2']
Processing outfall node: OF-V
Sorted Conduits: ['94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114'

In [ ]:
# Optimization: Create pipe-to-xsection mapping
pipe_to_xsection = {}
for xs_name, xs_data in xsections.items():
    if xs_data.link is not None and xs_data.link in inp["CONDUITS"]:
        pipe_to_xsection[xs_data.link] = xs_name  # Use link as key, xsection name as value

# Instead of creating a list, use itertools.product directly in the loop
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))
print(all_size_combinations)

for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)  # Create a copy of the initialized input

    # Create a dictionary to store sizes for all starting pipes in the current combination
    starting_pipe_sizes = {pipe: size for pipe, size in zip(all_starting_pipes, size_combination)}

    # Iterate through flow paths and apply sizes
    for outfall_node, flow_path_list in flow_paths.items():
        for flow_path in flow_path_list:
            flow_path.reverse()  # Reverse for upstream to downstream processing
            conduits_in_path = []  # Get conduits in this path
            for i in range(len(flow_path) - 1):
                from_node = flow_path[i]
                to_node = flow_path[i + 1]
                for conduit_name, conduit_data in conduit_connections.items():
                    if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                        conduits_in_path.append(conduit_name)
                        break

            # Find starting pipe in the flow path and apply its size to all conduits in the path
            for starting_pipe in all_starting_pipes:
                if starting_pipe in conduits_in_path:
                    starting_pipe_size = starting_pipe_sizes[starting_pipe]
                    for conduit_name in conduits_in_path:
                        xsection_name = pipe_to_xsection.get(conduit_name)
                        if xsection_name is not None:
                            xsection_object = modified_inp["XSECTIONS"][xsection_name]
                            xsection_object.height = starting_pipe_size
                            print(f"Pipe Name: {xsection_name}, Size: {xsection_object.height}")


    # Save the modified input file
    file_name = f"Optimized_Model1.inp"
    modified_inp.write_file(file_name)
    print(f"Modified input file saved as {file_name}")

[(0.76, 0.61, 1.5, 0.61, 1.8, 1.8, 1.8, 1.8, 0.76, 0.76, 0.45, 0.61, 1.8, 1.5, 1.05, 0.91, 0.91, 0.45, 0.76, 0.76, 0.61, 1.05)]
Modified input file saved as Optimized_Model1.inp


Downstream Optimization

In [ ]:
# --- Iterate downstream and optimize remaining conduits ---
new_file = SwmmInput("Optimized_Model1.inp")  # Load the optimized input file from Step 2
new_inp = copy.deepcopy(modified_inp)

# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits (modified to handle branches)
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit, considering branches
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Explore each branch

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths

# Get suitable sizes for starting pipes ONLY
velocities_per_pipe = {}
suitable_sizes_per_pipe = {}
ds_velocities = {}
ds_suitable_sizes = {}

# Track optimized conduits
optimized_conduits = set() # [CHECK IF THIS HAVE AN EFFECT]

# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break
        # Store the size of the preceding pipe (initialize with starting pipe size)
        # Assumes the starting pipes have been optimized in a previous step
        preceding_pipe_size = None
        if conduits_in_path:
             xsection_name = pipe_to_xsection.get(conduits_in_path[0])
             if xsection_name:
                 preceding_pipe_size = new_inp["XSECTIONS"][xsection_name].height

        for i in range(1, len(conduits_in_path)):
            conduit_name = conduits_in_path[i]

            # Check if the conduit has already been optimized [CHECK IF THIS HAVE AN EFFECT]
            if conduit_name in optimized_conduits:
                # Update preceding_pipe_size based on the already optimized size
                xsection_name = pipe_to_xsection.get(conduit_name)
                if xsection_name:
                    preceding_pipe_size = new_inp["XSECTIONS"][xsection_name].height
                continue  # Skip optimization for this conduit

            # (a) Calculate Peak Flow:
            peak_flow_value = peak_flow_solve("Optimized_Model1.inp", conduit_name)

            velocities_per_pipe = calculate_velocities(conduit_name, available_sizes, peak_flow_value)
            ds_velocities.update(velocities_per_pipe)
            all_velocities.update({conduit_name: ds_velocities[conduit_name]})
            suitable_sizes_per_pipe = find_suitable_sizes_with_velocity(ds_velocities, vel_min, vel_max)
            pipe_size = suitable_sizes_per_pipe.get(conduit_name)
            ds_suitable_sizes.update(suitable_sizes_per_pipe)

            # Enforce size constraint:
            optimal_size = preceding_pipe_size
            if suitable_sizes_per_pipe and pipe_size: # Check if suitable_sizes_per_pipe is not empty before applying the min function
              optimal_size = min(pipe_size[0], preceding_pipe_size)  # Ensure it's not larger than preceding pipe # min was comparing a dictionary and a float. The first element from the dictionary is selected to be compared
            else:
              optimal_size = preceding_pipe_size
            # Update the conduit's xsection:
            xsection_name = pipe_to_xsection.get(conduit_name)
            if xsection_name is not None:
                new_inp["XSECTIONS"][xsection_name].height = optimal_size
                optimized_conduits.add(conduit_name) # [CHECK IF THIS HAVE AN EFFECT]
            # Update preceding_pipe_size for the next iteration:
            preceding_pipe_size = optimal_size

file_name = f"Optimized_Model2.inp"
new_inp.write_file(file_name)
print(f"Modified input file saved as {file_name}")

Streaming output truncated to the last 5000 lines.
Conduit Name: 164, Suitable Sizes: [1.5]
Conduit Name: 165, Suitable Sizes: [1.5]
Conduit Name: 166, Suitable Sizes: [1.5]
Conduit Name: 167, Suitable Sizes: [1.5]
Conduit Name: 168, Suitable Sizes: [1.5]
Conduit Name: 169, Suitable Sizes: [1.5]
Conduit Name: 170, Suitable Sizes: [1.5]
Conduit Name: 171, Suitable Sizes: [1.5]
Conduit Name: 172, Suitable Sizes: [1.5]
Conduit Name: 173, Suitable Sizes: [1.5]
Conduit Name: 174, Suitable Sizes: [1.5]
Conduit Name: 175, Suitable Sizes: [1.5]
Conduit Name: 176, Suitable Sizes: [1.5]
Conduit Name: 177, Suitable Sizes: [1.5]
Conduit Name: 178, Suitable Sizes: [1.5]
Conduit Name: 179, Suitable Sizes: [1.5]
Conduit Name: 180, Suitable Sizes: [1.5]
Conduit Name: 181, Suitable Sizes: [1.5]
Conduit Name: 182, Suitable Sizes: [1.5]
Conduit Name: 183, Suitable Sizes: [1.5]
Conduit Name: 185, Suitable Sizes: [1.5]
Conduit Name: 186, Suitable Sizes: [1.5]
Conduit Name: 187, Suitable Sizes: [1.5]
Condui

# Step 3: Adjust Pipe Slopes Based on Adjusted Pipe Sizes

In [ ]:
# Get a list of all optimized input files from Step 2
adjusted_file = "Optimized_Model2.inp"
adjusted_inp = copy.deepcopy(new_inp)

# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths


# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits_in_path}")  # Print sorted conduits
        print(f"Junctions in Path: {junctions}")  # Print junctions in path

        # Reverse the conduits_in_path list before processing
        conduits_in_path.reverse()

        current_downstream_elevation = None  # Initialize for the first (last) conduit
        peak_flow_value = {}
        hydraulic_radius = {}
        pipe_size = {}
        new_slope = {}

        for conduit_name in conduits_in_path:  # Iterate in the correct order
            conduit_name_str = str(conduit_name)
            print(f"Currently Processing for Conduit: {conduit_name_str}")

            # Get upstream and downstream junction names
            upstream_junction = adjusted_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = adjusted_inp["CONDUITS"][conduit_name].to_node

            # Always use slope_min, regardless of original slope
            peak_flow_value = peak_flow_solve("Optimized_Model2.inp", conduit_name_str)
            print(f"Peak Flow: {peak_flow_value}")
            pipe_size = adjusted_inp["XSECTIONS"][conduit_name].height
            print(f"Pipe Size: {pipe_size}")
            hydraulic_radius = get_max_hydraulic_radius("Optimized_Model2.inp", conduit_name_str, pipe_size)
            print(f"R: {hydraulic_radius}")
            new_slope =  adjust_slope_with_mannings(pipe_size, peak_flow_value, mannings_n, hydraulic_radius )
            print(f"New Slope: {new_slope}")

            # If it's the first conduit (last in the original path),
            # use original downstream elevation from OUTFALLS or JUNCTIONS
            if conduit_name == conduits_in_path[0]:
                if downstream_junction in adjusted_inp["OUTFALLS"]:
                    current_downstream_elevation = adjusted_inp["OUTFALLS"][downstream_junction].elevation
                else:
                    current_downstream_elevation = adjusted_inp["JUNCTIONS"][downstream_junction].elevation

            # Otherwise, use the calculated upstream elevation of the previous conduit
            # as the current downstream elevation
            else:
                current_downstream_elevation = new_upstream_elevation  # Use the previously calculated value

            # Calculate new upstream elevation based on slope and current downstream elevation
            new_upstream_elevation = current_downstream_elevation + (new_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            adjusted_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            print(f"Downstream Elevation: {current_downstream_elevation}")
            print(f"Upstream Elevation: {new_upstream_elevation}")
            print(f"")

# Save the modified input file with a new name
file_name = f"Optimized_Model3.inp"
adjusted_inp.write_file(file_name)
print(f"Adjusted slopes/inverts and saved as {file_name}")

Processing outfall node: OF-X1
Sorted Conduits: ['44', '45', '46', '91']
Junctions in Path: ['X1', 'X2', 'X3', 'X4', 'OF-X1']
Currently Processing for Conduit: 91
Peak Flow: 0.3888011116282147
Pipe Size: 0.45
R: 0.12693358360102766
New Slope: 0.01583226902948722
Downstream Elevation: 22.01
Upstream Elevation: 22.28294831806836

Currently Processing for Conduit: 46
Peak Flow: 0.38878177892919646
Pipe Size: 0.45
R: 0.1125
New Slope: 0.018595106158848282
Downstream Elevation: 22.28294831806836
Upstream Elevation: 22.59906512276878

Currently Processing for Conduit: 45
Peak Flow: 0.3888367877019304
Pipe Size: 0.45
R: 0.1125
New Slope: 0.018600368577967614
Downstream Elevation: 22.59906512276878
Upstream Elevation: 22.971072494328133

Currently Processing for Conduit: 44
Peak Flow: 0.624757061271669
Pipe Size: 0.61
R: 0.1525
New Slope: 0.009479457644204744
Downstream Elevation: 22.971072494328133
Upstream Elevation: 23.161135620094438

Processing outfall node: OF-X2
Sorted Conduits: ['39', 

# Step 4: Optimize Pipe Slopes

In [ ]:
# Define maximum and minimum allowable slopes (adjust values as needed)
max_slope = 0.02  # Maximum allowable slope
min_slope = 0.01  # Minimum allowable slope
print(outfalls_data)
print(outfalls_data.keys())

study_inp = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp")
final_file = "Optimized_Model3.inp"
final_inp = copy.deepcopy(adjusted_inp)
junctions_data = study_inp['JUNCTIONS']
ground_slopes = {}

# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in study_inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths


# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits_in_path}")  # Print sorted conduits
        print(f"Junctions in Path: {junctions}")  # Print junctions in path

        # Reverse the conduits_in_path list before processing
        conduits_in_path.reverse()

        current_downstream_elevation = None  # Initialize for the first (last) conduit

        # Calculate ground slopes for all conduits before the main loop
        for conduit_name, conduit_data in study_inp["CONDUITS"].items():
            upstream_junction = conduit_data.from_node
            downstream_junction = conduit_data.to_node

            # Check if downstream node is an outfall
            if downstream_junction in study_inp["OUTFALLS"]:
                downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
                downstream_ground_elev = downstream_invert
            elif downstream_junction in study_inp["JUNCTIONS"]:
                downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
                downstream_ground_elev = downstream_invert + study_inp["JUNCTIONS"][downstream_junction].depth_max  # Assuming depth_max is 0 for outfalls
            else:
                print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
                downstream_invert = None  # or some default value

            upstream_invert = junctions_data[upstream_junction].elevation

            # Check if upstream node is an outfall (shouldn't happen, but handle for safety)
            if upstream_junction in study_inp["OUTFALLS"]:
                upstream_invert = study_inp["OUTFALLS"][upstream_junction].elevation  # This is unusual but handle it
                upstream_ground_elev = upstream_invert  # Assuming depth_max is 0 for outfalls
            elif upstream_junction in study_inp["JUNCTIONS"]:
                upstream_invert = study_inp["JUNCTIONS"][upstream_junction].elevation
                upstream_ground_elev = upstream_invert + study_inp["JUNCTIONS"][upstream_junction].depth_max  # Assuming depth_max is 0 for outfalls
            else:
                print(f"Warning: Upstream node '{upstream_junction}' not found in junctions or outfalls data.")
                upstream_invert = None  # or some default value
                upstream_ground_elev = None

            length = conduit_data.length

            if upstream_ground_elev is not None and downstream_ground_elev is not None:
                ground_slope = (upstream_ground_elev - downstream_ground_elev) / length
            else:
                ground_slope = None  # or some default value

            ground_slopes[conduit_name] = ground_slope
            print(f"Conduit: {conduit_name}, Ground Slope: {ground_slope}")

        for conduit_name in conduits_in_path:  # Iterate in the correct order
            conduit_name_str = str(conduit_name)
            print(f"Currently Processing for Conduit: {conduit_name_str}")

            if conduit_name == conduits_in_path[0]:

                if ground_slope < 0:
                    adjusted_slope = slope_min  # Use minimum slope
                else:
                    adjusted_slope = min(ground_slope, max_slope)


                if downstream_junction in adjusted_inp["OUTFALLS"]:
                    current_downstream_elevation = final_inp["OUTFALLS"][downstream_junction].elevation

                else:
                    current_downstream_elevation = final_inp["JUNCTIONS"][downstream_junction].elevation

                # Otherwise, use the calculated upstream elevation of the previous conduit
                # as the current downstream elevation

            else:
                current_downstream_elevation = new_upstream_elevation  # Use the previously calculated value

            # Calculate new upstream elevation based on slope and current downstream elevation
            new_upstream_elevation = current_downstream_elevation + (adjusted_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            final_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            print(f"Ground Slope: {ground_slope}")
            print(f"Adjusted Slope: {adjusted_slope}")
            print(f"Downstream Elevation: {current_downstream_elevation}")
            print(f"Upstream Elevation: {new_upstream_elevation}")
            print(f"")
            # Add the new logic to adjust junction depth if necessary


Streaming output truncated to the last 5000 lines.
Conduit: 325, Ground Slope: 0.004622496147919778
Conduit: 326, Ground Slope: -0.005924170616113618
Conduit: 327, Ground Slope: -0.015500000000000114
Conduit: 328, Ground Slope: -0.0129999999999999
Conduit: 329, Ground Slope: -0.005000000000000071
Conduit: 330, Ground Slope: 0.0025000000000000356
Conduit: 331, Ground Slope: 0.010999999999999944
Conduit: 332, Ground Slope: 0.015500000000000114
Conduit: 333, Ground Slope: 0.016499999999999914
Conduit: 334, Ground Slope: 0.01694915254237304
Conduit: 335, Ground Slope: 0.017021276595744317
Conduit: 336, Ground Slope: 0.005266622778143871
Conduit: 337, Ground Slope: -5.465713352000771e-16
Conduit: 338, Ground Slope: 0.36275862068965553
Conduit: 339, Ground Slope: 0.005000000000000071
Conduit: 340, Ground Slope: 0.005000000000000071
Conduit: 341, Ground Slope: 0.003333333333333381
Conduit: 342, Ground Slope: 0.005000000000000071
Conduit: 343, Ground Slope: 0.004999999999999893
Conduit: 344, G

In [ ]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file (re-using the existing logic)
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths

# Now, iterate through the junctions and adjust their depths
# We will iterate through the flow paths to ensure we process connected junctions
processed_junctions = set() # Use a set to avoid processing the same junction multiple times

for outfall_node, flow_path in flow_paths.items():
    for path in flow_path:
        # Reverse the path to process from upstream to downstream, visiting junctions in order
        path.reverse()

        for node in path:
            # Only process if the node is a junction and hasn't been processed yet
            if node in study_inp["JUNCTIONS"] and node in final_inp["JUNCTIONS"] and node not in processed_junctions:
                print(f"Adjusting depth for junction: {node}")

                # Get original and new junction data
                orig_junction_data = study_inp["JUNCTIONS"][node]
                final_junction_data = final_inp["JUNCTIONS"][node]

                # Calculate original and new ground elevations
                orig_ground_elev = orig_junction_data.elevation + orig_junction_data.depth_max
                print(f"Original Ground Elev: {orig_ground_elev}")

                new_ground_elev = final_junction_data.elevation + final_junction_data.depth_max
                print(f"New Ground Elev: {new_ground_elev}")

                # Apply the logic to adjust depth_max
                if orig_ground_elev > new_ground_elev:
                    # If original ground is higher, increase depth_max to reach original ground level
                    final_junction_data.depth_max = orig_ground_elev - final_junction_data.elevation
                    print(f"  Original ground ({orig_ground_elev:.2f}) > New ground ({new_ground_elev:.2f}). Setting new depth_max to {final_junction_data.depth_max:.2f}")
                elif orig_ground_elev < new_ground_elev:
                    # If new ground is higher, you might want to keep the original depth_max
                    # or set it to a minimum value if the new elevation makes the original depth unnecessary.
                    # Here, we'll set it to the original depth_max as an example.
                    final_junction_data.depth_max = final_junction_data.depth_max
                    print(f"Original ground ({orig_ground_elev:.2f}) < New ground ({new_ground_elev:.2f}). Setting new depth_max to final depth_max ({final_junction_data.depth_max:.2f})")
                else:
                    # If ground elevations are the same, keep the original depth_max
                    final_junction_data.depth_max = final_junction_data.depth_max
                    print(f"Original ground ({orig_ground_elev:.2f}) == New ground ({new_ground_elev:.2f}). Setting new depth_max to final depth_max ({final_junction_data.depth_max:.2f})")
                print(f"")
                processed_junctions.add(node) # Mark this junction as processed

# Save the modified input file with a new name
file_name = f"Final_Model.inp"
final_inp.write_file(file_name)
print(f"Optimized slopes/inverts and saved as {file_name}")

Adjusting depth for junction: X1
Original Ground Elev: 23.61
New Ground Elev: 24.44113562009444
Original ground (23.61) < New ground (24.44). Setting new depth_max to final depth_max (1.28)

Adjusting depth for junction: X2
Original Ground Elev: 23.42
New Ground Elev: 24.251072494328135
Original ground (23.42) < New ground (24.25). Setting new depth_max to final depth_max (1.28)

Adjusting depth for junction: X3
Original Ground Elev: 23.369999999999997
New Ground Elev: 23.88906512276878
Original ground (23.37) < New ground (23.89). Setting new depth_max to final depth_max (1.29)

Adjusting depth for junction: X4
Original Ground Elev: 23.32
New Ground Elev: 23.57294831806836
Original ground (23.32) < New ground (23.57). Setting new depth_max to final depth_max (1.29)

Adjusting depth for junction: X21/1
Original Ground Elev: 23.619999999999997
New Ground Elev: 24.35664847672295
Original ground (23.62) < New ground (24.36). Setting new depth_max to final depth_max (1.29)

Adjusting depth

# Step 5: Solving for Construction Costs

In [ ]:
# Load pipe prices data (assuming the file is named "pipe_prices.xlsx")
pipe_prices_file = r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Pipe_Prices.txt"
price_cost_lookup = {}

# Add a check to ensure the text file exists
if not os.path.exists(pipe_prices_file):
    print(f"Error: Pipe prices text file not found at: {pipe_prices_file}")
    # You might want to handle this error appropriately (e.g., exit)
else:
    print(f"Loading pipe prices from: {pipe_prices_file}")
    with open(pipe_prices_file, 'r') as f:
        for line in f:
            # Skip empty lines or lines starting with # (comments)
            if line.strip() and not line.strip().startswith('#'):
                try:
                    # Assuming space-separated values:
                    size_str, price_str = line.strip().split()
                    # If comma-separated, use: size_str, price_str = line.strip().split(',')

                    size = float(size_str)
                    price = float(price_str)
                    price_cost_lookup[size] = price
                except ValueError:
                    print(f"Warning: Skipping invalid line in pipe prices file: {line.strip()}")
                except IndexError:
                     print(f"Warning: Skipping incorrectly formatted line in pipe prices file: {line.strip()}")


# Print the loaded lookup dictionary to verify
print("Loaded Pipe Price Lookup:")
print(price_cost_lookup)

# Get a list of all slope-optimized input files from Step 4
study_inp = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp")
complete_file = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\RCP 4.5\(RCP4.5)50-Year_Final_Model\(RCP4.5)50-Year_Final_Model.inp")
complete_inp = copy.deepcopy(complete_file)

total_cost_before = 0
total_cost_after = 0

for conduit_name, conduit_data in study_inp["CONDUITS"].items():
    for xsection_name, xsection_data in study_inp["XSECTIONS"].items():
        if conduit_name == xsection_name:
           xs_name = conduit_name
           pipe_length = conduit_data.length
           pipe_size = xsection_data.height
           if pipe_size in price_cost_lookup:
              cost = price_cost_lookup[pipe_size]
              conduit_cost = cost * pipe_length
              total_cost_before += conduit_cost
              # print(f"Conduit Name: {conduit_name}")
              # print(f"Pipe Size = {pipe_size}")
              # print(f"Current cost: {total_cost_before}")
print(f"Cost Before Optimization: {total_cost_before}")

for conduit_name, conduit_data in final_inp["CONDUITS"].items():
    for xsection_name, xsection_data in final_inp["XSECTIONS"].items():
        if conduit_name == xsection_name:
           xs_name = conduit_name
           pipe_length = conduit_data.length
           pipe_size = xsection_data.height
          #  print(f"Conduit Name: {xs_name}")
          #  print(f"Pipe Length: {pipe_length}")
          #  print(f"Pipe Size: {pipe_size}")
           if pipe_size in price_cost_lookup:
              cost = price_cost_lookup[pipe_size]
              conduit_cost = cost * pipe_length
              total_cost_after += conduit_cost
              # print(f"Conduit Name: {conduit_name}")
              # print(f"Pipe Size = {pipe_size}")
              # print(f"Current cost: {total_cost_after}")
print(f"Cost After Optimization: {total_cost_after}")




Loading pipe prices from: C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Pipe_Prices.txt
Loaded Pipe Price Lookup:
{0.15: 300.0, 0.2: 375.0, 0.25: 425.0, 0.3: 639.5, 0.35: 950.0, 0.375: 758.23, 0.4: 1100.0, 0.45: 1009.93, 0.5: 1600.0, 0.61: 1750.0, 0.7: 2400.0, 0.76: 1607.0, 0.8: 3400.0, 0.9: 3900.0, 0.91: 2860.0, 1.05: 4725.0, 1.2: 4116.0, 1.5: 7131.1, 1.8: 9500.0}
Cost Before Optimization: 30742570.230999988
Cost After Optimization: 12599868.396999987


# Sensitivity Analysis


In [ ]:
# Get the file with the least construction cost from Step 5
least_cost_file = "slope_optimized_optimized_modified_input_starting_pipes_0.6_0.45_combination_0.3_0.3_0.45_0.3_0.3_0.3.inp" # Replace with your actual file name

# Load the least cost input file
least_cost_inp = SwmmInput(least_cost_file)

# Function to calculate average flow rate, size, and slope
def get_average_values(inp_file):
    """Calculates average flow rate, size, and slope for all pipes."""
    total_flow = 0
    total_size = 0
    total_slope = 0
    num_pipes = 0

    for conduit_name, conduit_data in inp_file["CONDUITS"].items():
        for xsection_name, xsection_data in inp["XSECTIONS"].items():
        # Calculate slope
          upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
          downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
          length = conduit_data.length
          slope = (upstream_invert - downstream_invert) / length

          # Get peak flow using peak_flow_solve
          peak_flow = peak_flow_solve(inp_file, conduit_name)

          # Get pipe size (assuming it's stored in the 'height' attribute)
          size = conduit_data.height

          total_flow += peak_flow
          total_size += size
          total_slope += slope
          num_pipes += 1

    avg_flow = total_flow / num_pipes if num_pipes else 0
    avg_size = total_size / num_pipes if num_pipes else 0
    avg_slope = total_slope / num_pipes if num_pipes else 0

    return avg_flow, avg_size, avg_slope

# Get average values for the least cost file
avg_flow, avg_size, avg_slope = get_average_values(least_cost_inp)
print(f"Average Flow Rate: {avg_flow}")
print(f"Average Pipe Size: {avg_size}")
print(f"Average Slope: {avg_slope}")

# Choose a pipe for sensitivity analysis (you can modify this)
target_pipe = "C1"  # Replace with the actual pipe name

# Function to get flow rate, size, and slope for a specific pipe
def get_pipe_values(inp_file, pipe_name):
    """Gets flow rate, size, and slope for a specific pipe."""
    conduit_data = inp_file["CONDUITS"][pipe_name]

    # Calculate slope
    upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
    downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
    length = conduit_data.length
    slope = (upstream_invert - downstream_invert) / length

    peak_flow = peak_flow_solve(inp_file, pipe_name) # peak_flow for the chosen conduit
    size = conduit_data.height

    return peak_flow, size, slope

# Get initial values for the target pipe
initial_flow, initial_size, initial_slope = get_pipe_values(least_cost_inp, target_pipe)
print(f"Flow Rate of {target_pipe}: {initial_flow}")
print(f"Size of {target_pipe}: {initial_size}")
print(f"Slope of {target_pipe}: {initial_slope}")


# # Define rainfall scenarios (replace with your actual scenarios)
# rainfall_scenarios = ["baseline", "scenario1", "scenario2"]

# # Sensitivity analysis loop for rainfall scenarios
# for scenario in rainfall_scenarios:
#     # Load modified input file for the current scenario
#     modified_inp_file = f"{scenario}.inp"  # Assuming you have .inp files for each scenario

#     # Create a copy of the least cost input file and modify with the given rainfall intensity
#     modified_inp = copy.deepcopy(least_cost_inp) # modified_inp is now the least cost input

#     # Placeholder - Update rainfall data in the copied least_cost_inp
#     # You need to implement your logic here to update the [TIMESERIES] section in modified_inp with data for the current scenario (modified_inp_file)

#     modified_inp.write_file("modified_least_cost.inp") # write modified inp to a temporary file to check the current peak flows
#     modified_inp = SwmmInput("modified_least_cost.inp") # call this so that the swmm library will update its parameters as well


#     # Get values for the target pipe after applying the rainfall scenario
#     flow, size, slope = get_pipe_values(modified_inp, target_pipe)

#     # Calculate the change in flow rate
#     flow_change = flow - initial_flow

#     # Print the results
#     print(f"\nScenario: {scenario}")
#     print(f"Flow Rate Change for {target_pipe}: {flow_change}")
#     print(f"Flow Rate of {target_pipe}: {flow}")
#     print(f"Size of {target_pipe}: {size}")
#     print(f"Slope of {target_pipe}: {slope}")

# Data Comparison

In [ ]:
study_inp = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp")
complete_inp = SwmmInput("Final_Model.inp")
conduit_info = []

for conduit_name, conduit_data in study_inp['CONDUITS'].items():
        upstream_junction = conduit_data.from_node
        downstream_junction = conduit_data.to_node
        pipe_name = conduit_name
        pipe_size_before = study_inp['XSECTIONS'][conduit_name].height
        pipe_size_after = complete_inp['XSECTIONS'][conduit_name].height

        upstream_invert_before = study_inp['JUNCTIONS'][upstream_junction].elevation
        upstream_inver_after = complete_inp['JUNCTIONS'][upstream_junction].elevation
        # Check if downstream node is an outfall
        if downstream_junction in outfalls_data:
            downstream_invert_before = study_inp['OUTFALLS'][downstream_junction].elevation  # Get elevation from outfalls_data
            downstream_invert_after = complete_inp['OUTFALLS'][downstream_junction].elevation
        elif downstream_junction in junctions_data:
            downstream_invert_before = study_inp['JUNCTIONS'][downstream_junction].elevation
            downstream_invert_after = complete_inp['JUNCTIONS'][downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
        else:
            print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
            downstream_invert_before = None  # or some default value
            downstream_invert_after = None

        print(f"Conduit Name: {conduit_name}")
        print(f"Upstream Junction: {upstream_junction}")
        print(f"Upstream Invert Before: {upstream_invert_before}")
        print(f"Upstream Invert After: {upstream_inver_after}")
        print(f"Downstream Junction: {downstream_junction}")
        print(f"Downstream Invert Before: {downstream_invert_before}")
        print(f"Downstream Invert After: {downstream_invert_after}")
        print(f"Original Pipe Size: {pipe_size_before}")
        print(f"Optimized Pipe Size: {pipe_size_after}")
        print(f"")
        conduit_info.append([conduit_name, upstream_junction, upstream_invert_before, upstream_inver_after, downstream_junction, downstream_invert_before, downstream_invert_after, pipe_size_before, pipe_size_after])

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Invert Before', 'Invert After', 'To Node', 'Invert Before', 'Invert After', 'Original Pipe Size', 'Optimized Pipe Size'])

conduit_df.to_excel("conduit_data_comparison.xlsx", index=False)

Conduit Name: 39
Upstream Junction: X21/1
Upstream Invert Before: 22.33
Upstream Invert After: 23.06665
Downstream Junction: X21/2
Downstream Invert Before: 22.2
Downstream Invert After: 22.77237
Original Pipe Size: 0.61
Optimized Pipe Size: 0.61

Conduit Name: 40
Upstream Junction: X23
Upstream Invert Before: 21.05
Upstream Invert After: 21.89447
Downstream Junction: X24
Downstream Invert Before: 21.01
Downstream Invert After: 21.60603
Original Pipe Size: 1.05
Optimized Pipe Size: 0.61

Conduit Name: 41
Upstream Junction: X24
Upstream Invert Before: 21.01
Upstream Invert After: 21.60603
Downstream Junction: X25
Downstream Invert Before: 20.97
Downstream Invert After: 21.3163
Original Pipe Size: 1.05
Optimized Pipe Size: 0.61

Conduit Name: 42
Upstream Junction: X25
Upstream Invert Before: 20.97
Upstream Invert After: 21.3163
Downstream Junction: X26
Downstream Invert Before: 20.93
Downstream Invert After: 21.06993
Original Pipe Size: 1.05
Optimized Pipe Size: 0.61

Conduit Name: 44
Up